In [ ]:
import pandas as pd
import psycopg2 
import os

run  `ssh -L 5433:localhost:5432 cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city -N` 
to establish ssh tunnel


or to forward socket:
`mkdir /tmp/sql-socket`
`ssh -L /tmp/sql-socket/.s.PGSQL.5432:/run/postgresql/.s.PGSQL.5432 -N cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city`
`psql -h /tmp/sql-socket -U cexplorer -d cdbsync`


In [ ]:
conn = psycopg2.connect(
    dbname='cdbsync',
    user='cexplorer',
    host='localhost',
    port=5433
)

cursor = conn.cursor()
cursor.execute('SELECT version();')
print(cursor.fetchone())

cursor.close()
conn.close()
